In [12]:
import xgboost as xgb
import catboost as ctb
import pickle
import pandas as pd
import numpy as np
from config.cicconfig import mapping
# from src.datasetconfig import mapping

In [14]:
# 加载XGBoost模型
clf = xgb.XGBClassifier()
clf.load_model('xgb_model.json')

# 加载CatBoost模型
cat_clf = ctb.CatBoostClassifier()
cat_clf.load_model('cat_model.cbm')

# 加载元模型
with open('meta_model.pkl', 'rb') as f:
    meta_model = pickle.load(f)

In [15]:
# clf.feature_names_in_

In [16]:
# 读取CSV文件
# data = pd.read_csv('06_22_2017-be-20170220-apps-com.hellochinese.pcap_ISCX.csv',encoding='gbk')  
data = pd.read_csv(path,encoding='gbk')  #检测捕获流量的原始文件夹
# data=preprocess_data(data,clf.feature_names_in_)
# 清理列名
# data.columns = data.columns.str.strip()


# 移除下面这两行，因为mapping变量未定义
data=data.rename(columns=mapping)
# data.drop(columns=['Fwd Header Length.1'], inplace=True)

# 确保列名格式正确


# print(data.info())
# data.columns

In [17]:
for col in data.columns:
    if col != 'Label': 
        try:
            series = pd.to_numeric(data[col], errors='coerce')
            series = series.replace([np.inf, -np.inf], np.nan)
            series = series.fillna(0).astype(int)
            data[col] = series
        except Exception as e:
            print(f"Error converting column {col}: {e}")

print("Data types after conversion:")

Data types after conversion:


In [18]:
# data.head(10)

In [19]:
X_new = data.drop(['Label'], axis=1, errors='ignore')
X_new.to_csv('output.csv', index=False)

In [20]:
# 获取基模型的预测概率
xgb_pred_proba = clf.predict_proba(X_new)
cat_pred_proba = cat_clf.predict_proba(X_new)

# 获取各模型的预测结果
xgb_pred = clf.predict(X_new)
cat_pred = cat_clf.predict(X_new)

# 拼接预测概率作为元模型的输入
meta_features = np.column_stack((xgb_pred_proba, cat_pred_proba))

# 使用元模型进行最终预测
final_pred = meta_model.predict(meta_features)


# 统计各模型恶性流量的数量
# 假设1为恶性流量标签
xgb_malicious_count = np.sum(xgb_pred == 1)
cat_malicious_count = np.sum(cat_pred == 1)
meta_malicious_count = np.sum(final_pred == 1)

# 输出各模型的恶性流量数量
print(f'XGBoost模型检测到的恶性流量数量: {xgb_malicious_count}')
print(f'CatBoost模型检测到的恶性流量数量: {cat_malicious_count}')
print(f'元模型检测到的恶性流量数量: {meta_malicious_count}')





XGBoost模型检测到的恶性流量数量: 2
CatBoost模型检测到的恶性流量数量: 0
元模型检测到的恶性流量数量: 1


In [ ]:
# 从最后一个代码单元格继续，创建一个包含预测结果的数据框
data_with_predictions = data.copy()
data_with_predictions['Prediction'] = final_pred

# 筛选出预测为恶意的流量 (label=1 表示恶意)
malicious_data = data_with_predictions[data_with_predictions['Prediction'] == 1]

# 打印恶意流量数据
print(f"恶意流量数量: {len(malicious_data)}")
print("\n恶意流量数据样例:")
print(malicious_data.head())

# 如果需要，可以保存这些恶意流量数据到CSV文件
malicious_data.to_csv('malicious_flows.csv', index=False)
print("\n恶意流量数据已保存到 'malicious_flows.csv'")

恶意流量数量: 1

恶意流量数据样例:
    Flow_ID  Source_IP  Source_Port  Destination_IP  Destination_Port  \
53        0          0        57179               0               443   

    Protocol  Timestamp  Flow_Duration  Total_Fwd_Packets  \
53         6          0       10212949                 10   

    Total_Backward_Packets  ...  Active_Mean  Active_Std  Active_Max  \
53                      12  ...       252967           0      252967   

    Active_Min  Idle_Mean  Idle_Std  Idle_Max  Idle_Min            Label  \
53      252967    9959982         0   9959982   9959982  NeedManualLabel   

    Prediction  
53           1  

[1 rows x 85 columns]

恶意流量数据已保存到 'malicious_flows.csv'
